In [2]:
import pandas as pd
import psycopg2
import pyodbc
import datetime
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [1]:
#Passwords
ONEVIEW_PASSWORD = ''
EDW_USERNAME = ''
EDW_PASSWORD = 'Slytherin10946!'
AIMS_USERNAME = 'vigrose'
AIMS_PASSWORD = 'DQA8K@T7pXEqEt'

In [3]:
#Standardize ME format function
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0','')
        num = '{:0>11}'.format(num)
        nums.append(num)
    return(nums)

In [4]:
#OneView connection
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password= ONEVIEW_PASSWORD)

In [5]:
#EDW connection
w = "DSN=PRDDW; UID={}; PWD={}".format(EDW_USERNAME, EDW_PASSWORD)
AMAEDW = pyodbc.connect(w)

In [6]:
#AIMS connection
s = "DSN=aims_prod; UID={}; PWD={}".format(AIMS_USERNAME, AIMS_PASSWORD)
informix = pyodbc.connect(s)

In [7]:
#Define universe
ov_me_sql = '''
    SELECT medical_education_number FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

C:\Users\vigrose\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
#Get race/ethnicity file from UDrive (AIMS table is limited access)
race = pd.read_csv('U:/Source Files/Data Analytics/Data-Science/Data/Masterfile_OneVIew/PhysicianRaceEthnicity_2_2022-05-02.csv')
race.medical_education_number = fix_me(race.medical_education_number)

In [9]:
#Get person type from AIMS
#FYI, querying from AIMS can often take a minute over the VPN 
type_query = \
    """
    SELECT
    ENTITY_ID,
    PERSON_TYPE
    FROM
    informix.PERSON_ROLE_ET
    WHERE
    ACTIVE_IND = 'Y'
    ;
    """
TYPE = pd.read_sql(con=informix, sql=type_query)
TYPE.columns = [x.upper() for x in TYPE.columns.values]
TYPE.ENTITY_ID = TYPE.ENTITY_ID.astype(str)

C:\Users\vigrose\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [11]:
#Get party IDs and MEs
PARTY_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME,
    P.UPDATE_TMSTP,
    P.ACTIVE_IND
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    """
PARTY_IDS = pd.read_sql(con=AMAEDW, sql=PARTY_QUERY)

C:\Users\vigrose\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [12]:
#Get entity ids
ENTITY_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ENTITY_ID
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 9
    """
ENTITY_IDS = pd.read_sql(con=AMAEDW, sql=ENTITY_QUERY)

C:\Users\vigrose\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
#Get person name information
NAME_QUERY = \
    """
    SELECT
    FIRST_NM, MIDDLE_NM, LAST_NM, SUFFIX, LABEL_NM, NM_TYPE, PARTY_ID
    FROM
    AMAEDW.PERSON_NM
    WHERE
    THRU_DT IS NULL
    AND
    NM_TYPE='LN'
    """
NAMES = pd.read_sql(con=AMAEDW, sql=NAME_QUERY)

In [ ]:
#get other person info
PERSON_QUERY = \
    """
    SELECT
    BIRTH_DT, BIRTH_CITY_NM, BIRTH_STATE_ID, BIRTH_COUNTRY_ID, GENDER_CD, MORTALITY_STS_CD, DEATH_DT, MED_EDU_NBR, PARTY_ID
    FROM
    AMAEDW.PERSON
    WHERE
    ACTIVE_IND = 'Y'
    """
PERSONS = pd.read_sql(con=AMAEDW, sql=PERSON_QUERY)

In [26]:
#Merge with pandas
ids = pd.merge(ENTITY_IDS, PARTY_IDS, on='PARTY_ID')
person_data = pd.merge(ids, ov_me, left_on='ME', right_on='medical_education_number', how='right')
person_data = pd.merge(person_data, TYPE, on='ENTITY_ID',how='left').drop_duplicates()
person_data = pd.merge(person_data, NAMES, on='PARTY_ID',how='left').drop_duplicates()
person_data = pd.merge(person_data, RACE, on='medical_education_number', how='left')
person_data = pd.merge(person_data, PERSONS, on='PARTY_ID',how='left').drop_duplicates()